In [1]:
import pandas as pd
from tqdm import tqdm
import re
from datetime import datetime

In [2]:
raw_df = pd.read_csv('./dataset/ready/raw_data_final.csv')
raw_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,customer_state,seller_zip_code_prefix,seller_city,seller_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,...,SP,9350,maua,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,1,87285b34884572647811a353c7ac498a,...,SP,9350,maua,SP,b46f1e34512b0f4c74a72398b03ca788,4,NaN,Deveriam embalar melhor o produto. A caixa vei...,2017-08-19 00:00:00,2017-08-20 15:16:36
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,1,87285b34884572647811a353c7ac498a,...,SP,9350,maua,SP,dc90f19c2806f1abba9e72ad3c350073,5,NaN,"Só achei ela pequena pra seis xícaras ,mais é ...",2017-08-08 00:00:00,2017-08-08 23:26:23
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13 00:00:00,1,87285b34884572647811a353c7ac498a,...,SC,9350,maua,SP,1bafb430e498b939f258b9c9dbdff9b1,3,NaN,NaN,2017-11-08 00:00:00,2017-11-10 19:52:38
4,8736140c61ea584cb4250074756d8f3b,ab8844663ae049fda8baf15fc928f47f,delivered,2017-08-10 13:35:55,2017-08-10 13:50:09,2017-08-11 13:52:35,2017-08-16 19:03:36,2017-08-23 00:00:00,1,b00a32a0b42fd65efb58a5822009f629,...,SP,9350,maua,SP,b8238c6515192f8129081e17dc57d169,5,NaN,"custo beneficio, simples de usar e rápido",2017-08-17 00:00:00,2017-08-21 12:43:27


In [3]:
print(raw_df.shape)
print(raw_df.drop_duplicates().shape)

(112372, 35)
(112372, 35)


In [4]:
raw_df.isna().sum() / len(raw_df)

order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_approved_at                0.000133
order_delivered_carrier_date     0.010536
order_delivered_customer_date    0.021002
order_estimated_delivery_date    0.000000
order_item_id                    0.000000
product_id                       0.000000
seller_id                        0.000000
shipping_limit_date              0.000000
price                            0.000000
freight_value                    0.000000
product_category_name            0.014221
product_name_lenght              0.014221
product_description_lenght       0.014221
product_photos_qty               0.014221
product_weight_g                 0.000160
product_length_cm                0.000160
product_height_cm                0.000160
product_width_cm                 0.000160
customer_unique_id               0.000000
customer_zip_code_prefix         0

In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112372 entries, 0 to 112371
Data columns (total 35 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       112372 non-null  object 
 1   customer_id                    112372 non-null  object 
 2   order_status                   112372 non-null  object 
 3   order_purchase_timestamp       112372 non-null  object 
 4   order_approved_at              112357 non-null  object 
 5   order_delivered_carrier_date   111188 non-null  object 
 6   order_delivered_customer_date  110012 non-null  object 
 7   order_estimated_delivery_date  112372 non-null  object 
 8   order_item_id                  112372 non-null  int64  
 9   product_id                     112372 non-null  object 
 10  seller_id                      112372 non-null  object 
 11  shipping_limit_date            112372 non-null  object 
 12  price                         

In [6]:
raw_df.describe()

,order_item_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,seller_zip_code_prefix,review_score
count,112372.000000,112372.000000,112372.000000,110774.000000,110774.000000,110774.000000,112354.000000,112354.000000,112354.000000,112354.000000,112372.000000,112372.000000,112372.000000
mean,1.196010,120.378962,19.977752,48.777583,786.793932,2.207124,2090.610882,30.152198,16.576811,23.001210,35131.880691,24451.942575,4.032473
std,0.691243,182.152386,15.781421,10.025179,651.609498,1.719787,3748.608093,16.139323,13.437792,11.707552,29894.587573,27607.504965,1.387849
min,1.000000,0.850000,0.000000,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000,1003.000000,1001.000000,1.000000
25%,1.000000,39.900000,13.070000,42.000000,348.000000,1.000000,300.000000,18.000000,8.000000,15.000000,11250.000000,6429.000000,4.000000
50%,1.000000,74.900000,16.250000,52.000000,601.000000,1.000000,700.000000,25.000000,13.000000,20.000000,24320.000000,13568.000000,5.000000
75%,1.000000,134.900000,21.150000,57.000000,985.000000,3.000000,1800.000000,38.000000,20.000000,30.000000,59063.000000,27930.000000,5.000000
max,21.000000,6735.000000,409.680000,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000,99990.000000,99730.000000,5.000000


In [7]:
empty_val_df = pd.DataFrame(raw_df.isna().sum(), columns=['num_empty_row']).reset_index()
empty_val_df.columns =  ['columns','num_empty_row']
empty_val_df['percentage'] = empty_val_df['num_empty_row'].apply(lambda x: round(x/len(raw_df)*100,2))
empty_val_df

,columns,num_empty_row,percentage
0,order_id,0,0.00
1,customer_id,0,0.00
2,order_status,0,0.00
3,order_purchase_timestamp,0,0.00
4,order_approved_at,15,0.01
5,order_delivered_carrier_date,1184,1.05
6,order_delivered_customer_date,2360,2.10
7,order_estimated_delivery_date,0,0.00
8,order_item_id,0,0.00
9,product_id,0,0.00


Ubah _data type_ kolom-kolom terkait tanggal dan jam agar bertipe datetime

In [10]:
raw_df['order_purchase_timestamp'] = pd.to_datetime(raw_df['order_purchase_timestamp'])
raw_df['order_approved_at'] = pd.to_datetime(raw_df['order_approved_at'])
raw_df['order_delivered_carrier_date'] = pd.to_datetime(raw_df['order_delivered_carrier_date'])
raw_df['order_delivered_customer_date'] = pd.to_datetime(raw_df['order_delivered_customer_date'])
raw_df['order_estimated_delivery_date'] = pd.to_datetime(raw_df['order_estimated_delivery_date'])
raw_df['shipping_limit_date'] = pd.to_datetime(raw_df['shipping_limit_date'])

Extract tahun, bulan, hari dan jam purchase

In [11]:
raw_df['order_purchase_timestamp'] = pd.to_datetime(raw_df['order_purchase_timestamp'])
raw_df['year_purchase'] = raw_df['order_purchase_timestamp'].dt.year
raw_df['month_purchase'] = raw_df['order_purchase_timestamp'].dt.month_name()
raw_df['day_purchase'] = raw_df['order_purchase_timestamp'].dt.day_name()
raw_df['hour_purchase'] = raw_df['order_purchase_timestamp'].dt.hour
raw_df[['order_purchase_timestamp','year_purchase','month_purchase','day_purchase','hour_purchase']]

,order_purchase_timestamp,year_purchase,month_purchase,day_purchase,hour_purchase
0,2017-10-02 10:56:33,2017,October,Monday,10
1,2017-08-15 18:29:31,2017,August,Tuesday,18
2,2017-08-02 18:24:47,2017,August,Wednesday,18
3,2017-10-23 23:26:46,2017,October,Monday,23
4,2017-08-10 13:35:55,2017,August,Thursday,13
...,...,...,...,...,...
112367,2018-07-01 10:23:10,2018,July,Sunday,10
112368,2017-03-11 19:51:36,2017,March,Saturday,19
112369,2018-07-24 09:46:27,2018,July,Tuesday,9
112370,2018-05-22 21:13:21,2018,May,Tuesday,21


Calculate process time dan delivery time

In [12]:
raw_df['process_time'] = raw_df['order_delivered_carrier_date'] - raw_df['order_approved_at']
raw_df['process_time_day'] = raw_df['process_time'].apply(lambda x: x.days)
raw_df['process_time_hour'] = raw_df['process_time'].apply(lambda x: round(x.total_seconds()/3600,2))

raw_df['real_delivery_time'] = raw_df['order_delivered_customer_date'] - raw_df['order_delivered_carrier_date']
raw_df['real_delivery_time_day'] = raw_df['real_delivery_time'].apply(lambda x: x.days)

In [13]:
state_df = pd.read_excel('./dataset/additional/state_code_name.xlsx')

state_dct = {k:v for k,v in zip(state_df['state_code'], state_df['state_name'])}

In [15]:
raw_df['customer_state_name'] = raw_df['customer_state'].apply(lambda x: state_dct.get(x))
raw_df['seller_state_name'] = raw_df['seller_state'].apply(lambda x: state_dct.get(x))
raw_df[['customer_state','customer_state_name','seller_state','seller_state_name']]

,customer_state,customer_state_name,seller_state,seller_state_name
0,SP,Sao Paulo,SP,Sao Paulo
1,SP,Sao Paulo,SP,Sao Paulo
2,SP,Sao Paulo,SP,Sao Paulo
3,SC,Santa Catarina,SP,Sao Paulo
4,SP,Sao Paulo,SP,Sao Paulo
...,...,...,...,...
112367,SP,Sao Paulo,SP,Sao Paulo
112368,SC,Santa Catarina,SC,Santa Catarina
112369,CE,Ceara,SP,Sao Paulo
112370,MG,Minas Gerais,SP,Sao Paulo


In [18]:
raw_df['product_category_name'] = raw_df['product_category_name'].fillna('Unknown')
raw_df['product_category_name'] = raw_df['product_category_name'].apply(lambda x: x.replace('_',' ').title())
raw_df[['product_category_name']]

,product_category_name
0,Utilidades Domesticas
1,Utilidades Domesticas
2,Utilidades Domesticas
3,Utilidades Domesticas
4,Bebes
...,...
112367,Construcao Ferramentas Iluminacao
112368,Automotivo
112369,Moveis Decoracao
112370,Moveis Decoracao


In [16]:
raw_df.to_csv('./dataset/ready/sales_data_processed.csv',index=False)